In [12]:
import xmltodict 
import pandas as pd
from os import listdir
import os


### COMPOSICIÓN DEL XML

<presentacion>
    <periodo>…</periodo>
    <nroPresentacion>…</nroPresentacion>
    <fechaPresentacion>…</fechaPresentacion>
    <empleado>…</empleado>
    <cargasFamilia>…</cargasFamilia>
    <ganLiqOtrosEmpEnt>…</ganLiqOtrosEmpEnt>
    <deducciones>…</deducciones>
    <retPerPagos>…</retPerPagos>
    <ajustes>…</ajustes>
    <datosAdicionales>…</datosAdicionales>
<presentacion>

Código Descripción
1 Cuotas Médico-Asistenciales
2 Primas de Seguro para el caso de muerte / riesgo de muerte
3 Donaciones
4 Intereses Préstamo Hipotecario
5 Gastos de Sepelio
7 Gastos Médicos y Paramédicos
8 Deducción del Personal Doméstico
11 Gastos de Representación e Intereses de Corredores y Viajantes de Comercio
21 Gastos de Adquisición de Indumentaria y Equipamiento para uso Exclusivo en el
22 Alquiler de Inmuebles destinados a casa habitación
23 Primas de Ahorro correspondientes a Seguros Mixtos
24 Aportes correspondientes a Planes de Seguro de Retiro Privados
32 Gastos de Educación

Deducciones que NO poseen periodos

9 Aporte a Sociedades de Garantía Recíproca
10 Vehiculos de Corredores y Viajantes de Comercio
25 Adquisición de Cuotapartes de Fondos Comunes de Inversión con fines de retiro
99 Otras Deducciones

In [13]:
def apertura_deducciones_por_mes(empleado, tipo_deduccion, mes_desde, mes_hasta, monto_mensual):
    '''
    Esta funcion devuelve un df con la cantidad de registros necesarios, segun el intervalo desde hasta (periodo) por cada deducción.
    '''
    datos_572 = [[empleado, tipo_deduccion, monto_mensual]]
    df_datos_572 = pd.DataFrame(datos_572, columns=["Cuil","Deduccion", "Importe"])

    cant_filas = (mes_hasta - mes_desde + 1)

    df_datos_572 = pd.concat([df_datos_572]*cant_filas, ignore_index=True)

    nueva_columna = []

    for i in range(14):
        if mes_desde <= i <= mes_hasta: 
            nueva_columna.append(i)

    for index, fila in df_datos_572.iterrows():
        mes = mes_desde
        if mes <= mes_hasta:
            df_datos_572["Mes"] = nueva_columna
            mes = mes_desde + 1

    return df_datos_572


In [14]:

CODIGOS_DEDUCC_MENSUAL = [1, 3, 4, 5, 8, 11, 21, 22, 32] #solamente dejar los que necesito que se procesen mensualmente

def deducciones_totales_por_mes(xml):

    empleado = xml["presentacion"]["empleado"]["cuit"]
    deducciones = xml["presentacion"]["deducciones"]["deduccion"]
    df_por_mes = pd.DataFrame([], columns=["Cuil","Deduccion", "Importe", "Mes"])
    if isinstance(deducciones, dict):
        deducciones = [deducciones]
    
    for i in deducciones:
        tipo_deduccion = int(i["@tipo"])
        if tipo_deduccion in CODIGOS_DEDUCC_MENSUAL:
            periodo = i["periodos"]["periodo"]
            
            #compruebo si es un diccionario y lo paso a listsa
            if isinstance(periodo, dict):
                periodo = [periodo]
            
            #para cada vaalor, determino los valores, ya sea que tengo una deduccion o muchas
            for j in periodo: 
                mes_desde = int(j["@mesDesde"])
                mes_hasta = int(j["@mesHasta"])
                monto_mensual = float(j["@montoMensual"])
                df = apertura_deducciones_por_mes(empleado, "D" + str(tipo_deduccion), mes_desde, mes_hasta, monto_mensual)
                df_por_mes = pd.concat([df_por_mes, df], ignore_index=True)            
        else:
            monto_anual = float(i["montoTotal"])
            df = apertura_deducciones_por_mes(empleado, "D" + str(tipo_deduccion), 13, 13, monto_anual)
            df_por_mes = pd.concat([df_por_mes, df], ignore_index=True)

    return df_por_mes


In [15]:
# path = "23278673804_2022_presentacion_006.xml"

# with open(path) as f:
#     xml = xmltodict.parse(f.read())

def cargas_por_mes(xml):

    empleado = xml["presentacion"]["empleado"]["cuit"]
    cargas = xml["presentacion"]["cargasFamilia"]["cargaFamilia"]
    df_por_mes_cargas = pd.DataFrame([], columns=["Cuil","Deduccion", "Importe", "Mes"])
    montos_cargas = {
        1 : 19621.44, 
        3 : 9895.16, 
        30 : 9895.16, 
        31 : 19790.33, 
        32: 19790.33
    }
        
    if isinstance(cargas, dict):
        cargas = [cargas]

    for i in cargas:
        parentesco = int(i["parentesco"])
        tipo_deduccion_carga = i["parentesco"]
        porcentaje_deduccion = int(i["porcentajeDeduccion"])
        mes_desde = int(i["mesDesde"])
        mes_hasta = int(i["mesHasta"])
        monto_mensual = montos_cargas[parentesco] * porcentaje_deduccion / 100
        df = apertura_deducciones_por_mes(empleado, "C" + tipo_deduccion_carga, mes_desde, mes_hasta, monto_mensual)
        df_por_mes_cargas = pd.concat([df_por_mes_cargas, df], ignore_index=True)  

    return(df_por_mes_cargas)



In [16]:

# path = "23165012879_2022_presentacion_011.xml"

# with open(path) as f:
#     xml = xmltodict.parse(f.read())
    
def df_otro_empleador(xml):

    empleado = xml["presentacion"]["empleado"]["cuit"]
    ingresos_oe = xml["presentacion"]["ganLiqOtrosEmpEnt"]["empEnt"]["ingresosAportes"]["ingAp"]
    df_otro_empleador = pd.DataFrame([], columns=["Cuil","Descripción", "Importe", "Mes"])

    if isinstance(ingresos_oe, dict):
        ingresos_oe = [ingresos_oe]

    for mes in ingresos_oe:
        for k in list(mes.keys()):
            if k == "@mes":
                continue 
            valor = float(mes[k])
            if valor != 0:
                datos_df_oe = [empleado, "OE-" + k, valor, int(mes["@mes"])]
                df_oe_new = pd.DataFrame([datos_df_oe], columns=["Cuil","Descripción", "Importe", "Mes"])
                df_otro_empleador = pd.concat([df_otro_empleador, df_oe_new], ignore_index=True)
        
    return(df_otro_empleador)


In [17]:
# path = "20179329973_2021_presentacion_005.xml"

# with open(path) as f:
#     xml = xmltodict.parse(f.read())



def df_ret_per_pago(xml):

    empleado = xml["presentacion"]["empleado"]["cuit"]
    rpp = xml["presentacion"]["retPerPagos"]["retPerPago"]
    df_por_rrp = pd.DataFrame([], columns=["Cuil","Deduccion", "Importe", "Mes"])
    
    if isinstance(rpp, dict):
        rpp = [rpp]
    
    for i in rpp:
        tipo_deduccion = int(i["@tipo"])            
        monto_mensual = float(i["montoTotal"])
        datos_df_rpp = [empleado, "RPP-" + str(tipo_deduccion), monto_mensual, 13]
        df = pd.DataFrame([datos_df_rpp], columns=["Cuil","Deduccion", "Importe", "Mes"])
        df_por_rrp = pd.concat([df_por_rrp, df], ignore_index=True) 

    return df_por_rrp

In [18]:
carpeta_xml = "../../bases-ganancias-2022/in/formularios572/"

In [19]:
def deducciones_finales(carpeta):

    listdir(carpeta_xml)
    df_deducciones = pd.DataFrame() #[], columns=["Cuil","Deduccion", "Importe", "Mes"])

    for a in listdir(carpeta_xml):
        path = carpeta_xml + "\\" + a

        with open(path) as f:
            xml = xmltodict.parse(f.read())

            if "deducciones" in xml["presentacion"]:
                df_deducc_mes_por_empleado = deducciones_totales_por_mes(xml)
                df_deducciones = pd.concat([df_deducc_mes_por_empleado, df_deducciones], ignore_index=True)

            if "cargasFamilia" in xml["presentacion"]:
                df_cargas_por_empleado_mes = cargas_por_mes(xml)
                df_deducciones = pd.concat([df_cargas_por_empleado_mes, df_deducciones], ignore_index=True)

            if "ganLiqOtrosEmpEnt" in xml["presentacion"]:
                df_ing_aport_otro_empleador = df_otro_empleador(xml)
                df_deducciones = pd.concat([df_ing_aport_otro_empleador, df_deducciones], ignore_index=True)

            if "retPerPagos" in xml["presentacion"]:
                df_rpp = df_ret_per_pago(xml)
                df_deducciones= pd.concat([df_rpp, df_deducciones], ignore_index=True)
                
    return(df_deducciones)


In [20]:
df = deducciones_finales(carpeta_xml)
df


,Cuil,Deduccion,Importe,Mes,Descripción
0,23278673804,RPP-27,33659.50,13,NaN
1,23278673804,RPP-28,1360.81,13,NaN
2,23278673804,C3,9895.16,1,NaN
3,23278673804,C3,9895.16,2,NaN
4,23278673804,C3,9895.16,3,NaN
...,...,...,...,...,...
190,20162547292,D99,40560.00,13,NaN
191,20162547292,D99,40560.00,13,NaN
192,20162547292,D99,47560.00,13,NaN
193,20162547292,D99,47560.00,13,NaN


In [21]:
#df.to_parquet('../572 web/572web.parquet', compression='brotli')


In [22]:
pd.read_parquet('../572 web/572web.parquet')

,Cuil,Deduccion,Importe,Mes,Descripción
0,23278673804,C3,9895.16,1,None
1,23278673804,C3,9895.16,2,None
2,23278673804,C3,9895.16,3,None
3,23278673804,C3,9895.16,4,None
4,23278673804,C3,9895.16,5,None
...,...,...,...,...,...
2029,20128767267,D8,25374.97,7,None
2030,20128767267,D8,27689.70,8,None
2031,20128767267,D8,30122.99,9,None
2032,20128767267,D8,32790.44,10,None
